In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [9]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [10]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [18]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.sort_values(by='sales_value', ascending=False).head(5)

,item_id,sales_value
56233,6534178,467993.62
56193,6533889,42645.75
29195,1029743,37981.91
56228,6534166,31298.96
35054,1082185,27291.02


In [17]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.sort_values(by='user_id', ascending=False).head(5)

,item_id,user_id
35054,1082185,2039
23846,981760,1584
25333,995242,1421
29195,1029743,1345
8056,840361,1319


In [19]:
popularity = data.groupby('item_id')['basket_id'].nunique().reset_index()
popularity.sort_values(by='basket_id', ascending=False).head(5)

,item_id,basket_id
35054,1082185,27362
56233,6534178,18364
29195,1029743,13455
25333,995242,11397
37719,1106523,9175


In [22]:
# В рекомендательных системах корректно разбивать 
# датасет по времени, а не случайно.
## ниже берутся последние 3 недели в качестве тестовой выборки.
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [26]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [30]:
# датафрейм с покупками юзеров на тестовом датасете 
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [31]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


In [33]:
# Random Recommendation

def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [35]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head()

CPU times: user 6.88 s, sys: 0 ns, total: 6.88 s
Wall time: 6.92 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9552984, 12427422, 1043084, 1038967, 925514]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1989299, 15507894, 9527187, 973311, 12132277]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1858995, 824392, 5586938, 1080455, 10356604]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[845397, 6533746, 850962, 12777301, 832105]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[846262, 1956421, 2439351, 9526323, 7409472]"


In [36]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)

    recs = popular.head(n).item_id
    return recs.tolist()

In [39]:
%%time

popular_recs = popularity_recommendation(data_train)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(3)

CPU times: user 215 ms, sys: 9.44 ms, total: 225 ms
Wall time: 228 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9552984, 12427422, 1043084, 1038967, 925514]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1989299, 15507894, 9527187, 973311, 12132277]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1858995, 824392, 5586938, 1080455, 10356604]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [65]:
# Random Weighted Recommendation

def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights.item_id
    weights = np.abs(items_weights.weight)
    recs = np.random.choice(items, p=weights, size=n, replace=False)
    
    return recs.tolist()

In [66]:
popular = data.groupby('item_id')['sales_value'].sum().reset_index()
popular.sort_values('sales_value', ascending=False, inplace=True)

popular['weight'] = popular['sales_value'].apply(lambda x: np.log(x) / popular.shape[0] if x > 0 else 0)

items_weights = popular[['item_id', 'weight']]
items_weights.loc[(items_weights['weight'] < 0), 'weight'] = 0
items_weights.describe()

/home/alina/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000030
std,5.178973e+06,0.000019
min,2.567100e+04,0.000000
25%,9.665830e+05,0.000014
50%,1.448516e+06,0.000027
75%,9.553042e+06,0.000043
max,1.802456e+07,0.000147


In [67]:
%%time

weighted_random_recommendation(items_weights)


ValueError: probabilities do not sum to 1